## Decision Tree for Spambase

In [ ]:
import numpy as np
import sys

### Get data

In [ ]:
def get_data():
    train_data = np.genfromtxt('./data/housing_train.txt')
    test_data = np.genfromtxt('./data/housing_test.txt')

    return train_data, test_data

In [ ]:
def get_data():
    train_data = np.genfromtxt('./data/housing_train.txt')
    test_data = np.genfromtxt('./data/housing_test.txt')

    return train_data, test_data


def normalize(data_set, train_len):
    # normalize data using shift/scale
    maxs = np.max(data_set, axis=0)
    mins = np.min(data_set, axis=0)

    maxs = maxs[:-1]
    mins = mins[:-1]

    for feature in range(len(data_set[0])-1):
        for entry in data_set:
            entry[feature] = (entry[feature] - mins[feature]) / (maxs[feature] - mins[feature])

    return data_set[:train_len], data_set[train_len:]

def get_thresholds(dataset):

    ts = []
    for feature in range(len(dataset[0])-1):
        t = []
        for entry in range(len(dataset) - 1):
            t.append((dataset[entry][feature] + dataset[entry+1][feature])/2)
        ts.append(t)

    return np.array(ts)


def get_prediction(dataset):
    return np.mean(dataset, axis=0)